# Import Libraries

In [1]:
import os

import pandas
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

import modules.prepare as myPrepare
import modules.preprocess as myPrep

# Global Variables

In [2]:
# path
BASE_PATH: str = os.path.abspath('./')
DATASET_PATH: str = BASE_PATH + '/data'

# file name
TRAIN_DATA_PATH: str = DATASET_PATH + '/train.csv'
TEST_DATA_PATH: str = DATASET_PATH + '/test.csv'
SUBMISSION_PATH: str = DATASET_PATH + '/gender_submission.csv'

# dataset
TARGET_COLUMN = 'Survived'

# Get Dataset

In [3]:
# type: pandas.DataFrame
trainData = myPrepare.getRawData(TRAIN_DATA_PATH)
trainData.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


# Preprocess Dataset

In [4]:
# type: pandas.DataFrame
trainDataPrep = myPrep.preprocess(data = trainData, targetColumn = TARGET_COLUMN)
trainDataPrep.head(10)

,Survived,Pclass,Age,Fare
0,0,3,22.000000,7.2500
1,1,1,38.000000,71.2833
2,1,3,26.000000,7.9250
3,1,1,35.000000,53.1000
4,0,3,35.000000,8.0500
5,0,3,29.699118,8.4583
6,0,1,54.000000,51.8625
7,0,3,2.000000,21.0750
8,1,3,27.000000,11.1333
9,1,2,14.000000,30.0708


# Build Model

In [5]:
# Divide train data into train & valid data
# type: numpy.ndarray
X = trainDataPrep.drop('Survived', axis = 1).values
y = trainDataPrep['Survived'].values
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size = 0.8, random_state = 42)

In [6]:
model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_valid)

In [7]:
accuracy: float = accuracy_score(y_valid, y_pred)
accuracy

0.7374301675977654

# Submit

In [8]:
# type: pandas.DataFrame
testData = myPrepare.getRawData(TEST_DATA_PATH)
testData.head(10)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
5,897,3,"Svensson, Mr. Johan Cervin",male,14.0,0,0,7538,9.2250,NaN,S
6,898,3,"Connolly, Miss. Kate",female,30.0,0,0,330972,7.6292,NaN,Q
7,899,2,"Caldwell, Mr. Albert Francis",male,26.0,1,1,248738,29.0000,NaN,S
8,900,3,"Abrahim, Mrs. Joseph (Sophie Halaut Easu)",female,18.0,0,0,2657,7.2292,NaN,C
9,901,3,"Davies, Mr. John Samuel",male,21.0,2,0,A/4 48871,24.1500,NaN,S


In [9]:
# type: pandas.DataFrame
testDataPrep = myPrep.preprocess(data = testData, targetColumn = TARGET_COLUMN)
testDataPrep.head(10)

,Pclass,Age,Fare
0,3,34.5,7.8292
1,3,47.0,7.0000
2,2,62.0,9.6875
3,3,27.0,8.6625
4,3,22.0,12.2875
5,3,14.0,9.2250
6,3,30.0,7.6292
7,2,26.0,29.0000
8,3,18.0,7.2292
9,3,21.0,24.1500


In [10]:
pred = model.predict(testDataPrep)

In [11]:
submitData = pandas.DataFrame({
    'PassengerId': testData['PassengerId'],
    'Survived': pred
})

In [12]:
submitData.head(10)

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
5,897,0
6,898,0
7,899,0
8,900,0
9,901,0


In [13]:
submitData.to_csv(SUBMISSION_PATH, index = False)